# Fields for tournament games

# Informational fields:
* date
* season
* team
* opponent
* win/loss

# Predictor fields
* regular_season_win_pct - just conference and non-conference games
* opponent_regular_season_win_pct
* season_win_pct - includes conference tourney games
* opponent_season_win_pct
* season_ppg - includes conference tourney games
* opponent_season_ppg
* season_ppg_allowed - includes conference tourney games
* opponent_season_ppg_allowed
* last_10_games_win_pct - includes conference tourney games
* opponent_last_10_games_win_pct
* last_5_games_win_pct - includes conference tourney games
* opponent_last_5_games_win_pct
* vs_tourney_team_win_pct - includes conference tourney games
* opponent_vs_tourney_team_win_pct
* games_played_vs_tourney_teams - includes conference tourney games
* opponent_games_played_vs_tourney_teams
* season_adj_offense - includes conference tourney games
* opponent_season_adj_offense
* vs_tourney_team_adj_offense - includes conference tourney games
* opponent_vs_tourney_team_adj_offense
* season_adj_defense - includes conference tourney games
* opponent_season_adj_defense
* vs_tourney_team_adj_defense - includes conference tourney games
* opponent_vs_tourney_team_adj_defense
* season_offensive_efficiency - includes conference tourney games
* opponent_season_offensive_efficiency
* vs_tourney_team_offensive_efficiency - includes conference tourney games
* opponent_vs_tourney_team_offensive_efficiency
* season_effective_field_goal_pct - includes conference tourney games
* opponent_season_effective_field_goal_pct
* vs_tourney_team_effective_field_goal_pct - includes conference tourney games
* opponent_vs_tourney_team_effective_field_goal_pct
* season_turnover_pct - includes conference tourney games
* opponent_season_turnover_pct
* vs_tourney_team_turnover_pct - includes conference tourney games
* opponent_vs_tourney_team_turnover_pct
* season_off_reb_pct - includes conference tourney games
* opponent_season_off_reb_pct
* vs_tourney_team_off_reb_pct - includes conference tourney games
* opponent_vs_tourney_team_off_reb_pct
* season_ft_rate - includes conference tourney games
* opponent_season_ft_rate
* vs_tourney_team_ft_rate - includes conference tourney games
* opponent_vs_tourney_team_ft_rate
* season_allowed_offensive_efficiency - includes conference tourney games
* opponent_season_allowed_offensive_efficiency
* vs_tourney_team_allowed_offensive_efficiency - includes conference tourney games
* opponent_vs_tourney_team_allowed_offensive_efficiency
* season_allowed_effective_field_goal_pct - includes conference tourney games
* opponent_season_allowed_effective_field_goal_pct
* vs_tourney_team_allowed_effective_field_goal_pct - includes conference tourney games
* opponent_vs_tourney_team_allowed_effective_field_goal_pct
* season_allowed_turnover_pct - includes conference tourney games
* opponent_season_allowed_turnover_pct
* vs_tourney_team_allowed_turnover_pct - includes conference tourney games
* opponent_vs_tourney_team_allowed_turnover_pct
* season_allowed_off_reb_pct - includes conference tourney games
* opponent_season_allowed_off_reb_pct
* vs_tourney_team_allowed_off_reb_pct - includes conference tourney games
* opponent_vs_tourney_team_allowed_off_reb_pct
* season_allowed_ft_rate - includes conference tourney games
* opponent_season_allowed_ft_rate
* vs_tourney_team_allowed_ft_rate - includes conference tourney games
* opponent_vs_tourney_team_allowed_ft_rate
* season_allowed_adj_offense - includes conference tourney games
* opponent_season_allowed_adj_offense
* vs_tourney_team_allowed_adj_offense - includes conference tourney games
* opponent_vs_tourney_team_allowed_adj_offense
* season_allowed_adj_defense - includes conference tourney games
* opponent_season_allowed_adj_defense
* vs_tourney_team_allowed_adj_defense - includes conference tourney games
* opponent_vs_tourney_team_allowed_adj_defense

In [238]:
import os
import pandas as pd
import sqlalchemy
import sklearn
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [239]:
all_games = pd.read_sql("select * from sports_data.game_stats_v2", conn)

In [240]:
tourney_games = all_games[all_games["game_type"] == "Post-Season Tourney"]
tourney_games = tourney_games[tourney_games['location'] == 'N']

In [241]:
tourney_games.columns

Index(['date', 'game_type', 'team', 'conference', 'opponent', 'location',
       'result', 'adj_offense', 'adj_defense', 'offensive_efficiency',
       'effective_fg_percent', 'turnover_percent', 'offensive_rebound_percent',
       'free_throw_rate', 'opponent_offensive_efficiency',
       'opponent_effective_fg_percent', 'opponent_turnover_percent',
       'opponent_offensive_rebound_percent', 'opponent_free_throw_rate',
       'opponent_conference', 'season', 'Win_Loss', 'score', 'opponent_score'],
      dtype='object')

In [242]:
tourney_games = tourney_games[['date', 'season', 'team', 'opponent', 'Win_Loss']]

In [243]:
all_games['game_type'].unique()

array(['Non-Conference', 'Conference', 'Conference Tourney',
       'Post-Season Tourney'], dtype=object)

In [244]:
regular_season_games = all_games[all_games["game_type"].isin(['Non-Conference', 'Conference'])]
pre_tourney_games = all_games[all_games["game_type"].isin(['Non-Conference', 'Conference', 'Conference Tourney'])]
last_5_non_tourney = all_games[all_games['game_type'] != 'Post-Season Tourney'].sort_values('date').groupby(['team', 'season']).tail(5).sort_values(['team', 'season', 'date'])
last_10_non_tourney = all_games[all_games['game_type'] != 'Post-Season Tourney'].sort_values('date').groupby(['team', 'season']).tail(10).sort_values(['team', 'season', 'date'])
all_games['opp_season'] = all_games.apply(lambda x: f"{x['opponent']}_{x['season']}", axis=1)
tourney_teams = tourney_games.apply(lambda x: f"{x['team']}_{x['season']}", axis=1).unique()
games_vs_tourney_teams = all_games[all_games['opp_season'].isin(tourney_teams)]

In [245]:
working = regular_season_games.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['regular_season_win_pct'] = working['W'] / (working['W'] + working['L'])
regular_season_win_pct = working[['team', 'season', 'regular_season_win_pct']]

In [246]:
tourney_games = tourney_games.merge(regular_season_win_pct, on=['team', 'season'], how='left')

In [247]:
regular_season_win_pct.rename(columns={'regular_season_win_pct': 'opponent_regular_season_win_pct', 'team':'opponent'}, inplace=True)

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\1859462520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regular_season_win_pct.rename(columns={'regular_season_win_pct': 'opponent_regular_season_win_pct', 'team':'opponent'}, inplace=True)


In [248]:
tourney_games

,date,season,team,opponent,Win_Loss,regular_season_win_pct
0,3/17/23,2023,Arizona St.,TCU,L,0.645161
1,3/17/23,2023,TCU,Arizona St.,W,0.645161
2,3/23/23,2023,Arkansas,Connecticut,L,0.612903
3,3/23/23,2023,Connecticut,Arkansas,W,0.774194
4,3/18/23,2023,Arkansas,Kansas,W,0.612903
...,...,...,...,...,...,...
1237,3/19/15,2015,Wofford,Arkansas,L,0.793103
1238,3/20/15,2015,Northern Iowa,Wyoming,W,0.900000
1239,3/20/15,2015,Wyoming,Northern Iowa,L,0.689655
1240,3/26/15,2015,Arizona,Xavier,W,0.903226


In [249]:
tourney_games = tourney_games.merge(regular_season_win_pct, on=['opponent', 'season'], how='left')

In [250]:
working = pre_tourney_games.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['season_win_pct'] = working['W'] / (working['W'] + working['L'])
season_win_pct = working[['team', 'season', 'season_win_pct']]

In [251]:
tourney_games = tourney_games.merge(season_win_pct, on=['team', 'season'], how='left')

In [252]:
season_win_pct.rename(columns={'season_win_pct': 'opponent_season_win_pct', 'team':'opponent'}, inplace=True)

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\1093291026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_win_pct.rename(columns={'season_win_pct': 'opponent_season_win_pct', 'team':'opponent'}, inplace=True)


In [253]:
tourney_games = tourney_games.merge(season_win_pct, on=['opponent', 'season'], how='left')

In [254]:
pre_tourney_games['score'] = pre_tourney_games['score'].astype(int)
season_ppg = pre_tourney_games[['team', 'season','score']].groupby(['team', 'season']).mean().reset_index().rename(columns={'score': 'season_ppg'})

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\2704502709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_tourney_games['score'] = pre_tourney_games['score'].astype(int)


In [255]:
tourney_games = tourney_games.merge(season_ppg, on=['team', 'season'], how='left')

In [256]:
season_ppg.rename(columns={'season_ppg': 'opponent_season_ppg', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg, on=['opponent', 'season'], how='left')

In [257]:
pre_tourney_games['opponent_score'] = pre_tourney_games['opponent_score'].astype(int)
season_ppg_allowed = pre_tourney_games[['team', 'season','opponent_score']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_score': 'season_ppg_allowed'})

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\2874491460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_tourney_games['opponent_score'] = pre_tourney_games['opponent_score'].astype(int)


In [258]:
season_ppg_std = pre_tourney_games[['team', 'season','score']].groupby(['team', 'season']).std().reset_index().rename(columns={'score': 'season_ppg_std'})
tourney_games = tourney_games.merge(season_ppg_std, on=['team', 'season'], how='left')
season_ppg_std.rename(columns={'season_ppg_std': 'opponent_season_ppg_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg_std, on=['opponent', 'season'], how='left')

In [259]:
season_ppg_allowed_std = pre_tourney_games[['team', 'season','opponent_score']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_score': 'season_ppg_allowed_std'})
tourney_games = tourney_games.merge(season_ppg_allowed_std, on=['team', 'season'], how='left')
season_ppg_allowed_std.rename(columns={'season_ppg_allowed_std': 'opponent_season_ppg_allowed_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg_allowed_std, on=['opponent', 'season'], how='left')

In [260]:
tourney_games = tourney_games.merge(season_ppg_allowed, on=['team', 'season'], how='left')

In [261]:
season_ppg_allowed.rename(columns={'season_ppg_allowed': 'opponent_season_ppg_allowed', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg_allowed, on=['opponent', 'season'], how='left')

In [262]:
working = last_10_non_tourney.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['last_10_win_pct'] = working['W'] / (working['W'] + working['L'])
last_10_win_pct = working[['team', 'season', 'last_10_win_pct']]

In [263]:
tourney_games = tourney_games.merge(last_10_win_pct, on=['team', 'season'], how='left')

In [264]:
last_10_win_pct.rename(columns={'last_10_win_pct': 'opponent_last_10_win_pct', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_win_pct, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\1411312810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_win_pct.rename(columns={'last_10_win_pct': 'opponent_last_10_win_pct', 'team':'opponent'}, inplace=True)


In [265]:
working = last_5_non_tourney.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['last_5_win_pct'] = working['W'] / (working['W'] + working['L'])
last_5_win_pct = working[['team', 'season', 'last_5_win_pct']]

In [266]:
tourney_games = tourney_games.merge(last_5_win_pct, on=['team', 'season'], how='left')

In [267]:
last_5_win_pct.rename(columns={'last_5_win_pct': 'opponent_last_5_win_pct', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_5_win_pct, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\489938241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_5_win_pct.rename(columns={'last_5_win_pct': 'opponent_last_5_win_pct', 'team':'opponent'}, inplace=True)


In [268]:
working = games_vs_tourney_teams.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['vs_tourney_team_win_pct'] = working['W'] / (working['W'] + working['L'])
vs_tourney_team_win_pct = working[['team', 'season', 'vs_tourney_team_win_pct']]

In [269]:
tourney_games = tourney_games.merge(vs_tourney_team_win_pct, on=['team', 'season'], how='left')

In [270]:
vs_tourney_team_win_pct.rename(columns={'vs_tourney_team_win_pct': 'opponent_vs_tourney_team_win_pct', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_win_pct, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\2986988654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vs_tourney_team_win_pct.rename(columns={'vs_tourney_team_win_pct': 'opponent_vs_tourney_team_win_pct', 'team':'opponent'}, inplace=True)


In [271]:
working = games_vs_tourney_teams.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['games_played_vs_tourney_teams'] = (working['W'] + working['L'])
games_played_vs_tourney_teams = working[['team', 'season', 'games_played_vs_tourney_teams']]

In [272]:
tourney_games = tourney_games.merge(games_played_vs_tourney_teams, on=['team', 'season'], how='left')

In [273]:
games_played_vs_tourney_teams.rename(columns={'games_played_vs_tourney_teams': 'opponent_games_played_vs_tourney_teams', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(games_played_vs_tourney_teams, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_24184\3857983212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_played_vs_tourney_teams.rename(columns={'games_played_vs_tourney_teams': 'opponent_games_played_vs_tourney_teams', 'team':'opponent'}, inplace=True)


In [153]:
pre_tourney_games.columns

Index(['date', 'game_type', 'team', 'conference', 'opponent', 'location',
       'result', 'adj_offense', 'adj_defense', 'offensive_efficiency',
       'effective_fg_percent', 'turnover_percent', 'offensive_rebound_percent',
       'free_throw_rate', 'opponent_offensive_efficiency',
       'opponent_effective_fg_percent', 'opponent_turnover_percent',
       'opponent_offensive_rebound_percent', 'opponent_free_throw_rate',
       'opponent_conference', 'season', 'Win_Loss', 'score', 'opponent_score'],
      dtype='object')

In [274]:
season_adj_offense = pre_tourney_games[['team', 'season','adj_offense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_offense': 'season_adj_offense'})

In [275]:
tourney_games = tourney_games.merge(season_adj_offense, on=['team', 'season'], how='left')

In [276]:
season_adj_offense.rename(columns={'season_adj_offense': 'opponent_season_adj_offense', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_offense, on=['opponent', 'season'], how='left')

In [277]:
season_adj_offense_std = pre_tourney_games[['team', 'season','adj_offense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_offense': 'season_adj_offense_std'})
tourney_games = tourney_games.merge(season_adj_offense_std, on=['team', 'season'], how='left')
season_adj_offense_std.rename(columns={'season_adj_offense_std': 'opponent_season_adj_offense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_offense_std, on=['opponent', 'season'], how='left')

In [278]:
season_adj_defense = pre_tourney_games[['team', 'season','adj_defense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_defense': 'season_adj_defense'})

In [279]:
tourney_games = tourney_games.merge(season_adj_defense, on=['team', 'season'], how='left')

In [280]:
season_adj_defense.rename(columns={'season_adj_defense': 'opponent_season_adj_defense', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_defense, on=['opponent', 'season'], how='left')

In [281]:
season_adj_defense_std = pre_tourney_games[['team', 'season','adj_defense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_defense': 'season_adj_defense_std'})
tourney_games = tourney_games.merge(season_adj_defense_std, on=['team', 'season'], how='left')
season_adj_defense_std.rename(columns={'season_adj_defense_std': 'opponent_season_adj_defense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_defense_std, on=['opponent', 'season'], how='left')

In [282]:
vs_tourney_team_adj_offense = games_vs_tourney_teams[['team', 'season','adj_offense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_offense': 'vs_tourney_team_adj_offense'})

In [283]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense, on=['team', 'season'], how='left')

In [284]:
vs_tourney_team_adj_offense.rename(columns={'vs_tourney_team_adj_offense': 'opponent_vs_tourney_team_adj_offense', 'team':'opponent'}, inplace=True)

In [285]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense, on=['opponent', 'season'], how='left')

In [286]:
vs_tourney_team_adj_offense_std = games_vs_tourney_teams[['team', 'season','adj_offense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_offense': 'vs_tourney_team_adj_offense_std'})
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense_std, on=['team', 'season'], how='left')
vs_tourney_team_adj_offense_std.rename(columns={'vs_tourney_team_adj_offense_std': 'opponent_vs_tourney_team_adj_offense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense_std, on=['opponent', 'season'], how='left')

In [287]:
vs_tourney_team_adj_defense = games_vs_tourney_teams[['team', 'season','adj_defense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_defense': 'vs_tourney_team_adj_defense'})

In [288]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense, on=['team', 'season'], how='left')

In [289]:
vs_tourney_team_adj_defense.rename(columns={'vs_tourney_team_adj_defense': 'opponent_vs_tourney_team_adj_defense', 'team':'opponent'}, inplace=True)

In [290]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense, on=['opponent', 'season'], how='left')

In [291]:
vs_tourney_team_adj_defense_std = games_vs_tourney_teams[['team', 'season','adj_defense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_defense': 'vs_tourney_team_adj_defense_std'})
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense_std, on=['team', 'season'], how='left')
vs_tourney_team_adj_defense_std.rename(columns={'vs_tourney_team_adj_defense_std': 'opponent_vs_tourney_team_adj_defense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense_std, on=['opponent', 'season'], how='left')

In [292]:
season_offensive_efficiency = pre_tourney_games[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_efficiency': 'season_offensive_efficiency'})

In [293]:
tourney_games = tourney_games.merge(season_offensive_efficiency, on=['team', 'season'], how='left')

In [294]:
season_offensive_efficiency.rename(columns={'season_offensive_efficiency': 'opponent_season_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_efficiency, on=['opponent', 'season'], how='left')

In [296]:
season_offensive_efficiency_std = pre_tourney_games[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_efficiency': 'season_offensive_efficiency_std'})
tourney_games = tourney_games.merge(season_offensive_efficiency_std, on=['team', 'season'], how='left')
season_offensive_efficiency_std.rename(columns={'season_offensive_efficiency_std': 'opponent_season_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [297]:
vs_tourney_team_offensive_efficiency = games_vs_tourney_teams[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_efficiency': 'vs_tourney_team_offensive_efficiency'})

In [298]:
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency, on=['team', 'season'], how='left')

In [299]:
vs_tourney_team_offensive_efficiency.rename(columns={'vs_tourney_team_offensive_efficiency': 'opponent_vs_tourney_team_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency, on=['opponent', 'season'], how='left')

In [300]:
vs_tourney_team_offensive_efficiency_std = games_vs_tourney_teams[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_efficiency': 'vs_tourney_team_offensive_efficiency_std'})
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency_std, on=['team', 'season'], how='left')
vs_tourney_team_offensive_efficiency_std.rename(columns={'vs_tourney_team_offensive_efficiency_std': 'opponent_vs_tourney_team_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [301]:
season_effective_fg_percent = pre_tourney_games[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'effective_fg_percent': 'season_effective_fg_percent'})

In [302]:
tourney_games = tourney_games.merge(season_effective_fg_percent, on=['team', 'season'], how='left')

In [303]:
season_effective_fg_percent.rename(columns={'season_effective_fg_percent': 'opponent_season_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_effective_fg_percent, on=['opponent', 'season'], how='left')

In [304]:
season_effective_fg_percent_std = pre_tourney_games[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'effective_fg_percent': 'season_effective_fg_percent_std'})
tourney_games = tourney_games.merge(season_effective_fg_percent_std, on=['team', 'season'], how='left')
season_effective_fg_percent_std.rename(columns={'season_effective_fg_percent_std': 'opponent_season_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [305]:
vs_tourney_team_effective_fg_percent = games_vs_tourney_teams[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'effective_fg_percent': 'vs_tourney_team_effective_fg_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent, on=['team', 'season'], how='left')
vs_tourney_team_effective_fg_percent.rename(columns={'vs_tourney_team_effective_fg_percent': 'opponent_vs_tourney_team_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent, on=['opponent', 'season'], how='left')

In [306]:
vs_tourney_team_effective_fg_percent_std = games_vs_tourney_teams[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'effective_fg_percent': 'vs_tourney_team_effective_fg_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_effective_fg_percent_std.rename(columns={'vs_tourney_team_effective_fg_percent_std': 'opponent_vs_tourney_team_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [307]:
season_turnover_percent = pre_tourney_games[['team', 'season','turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'turnover_percent': 'season_turnover_percent'})
tourney_games = tourney_games.merge(season_turnover_percent, on=['team', 'season'], how='left')
season_turnover_percent.rename(columns={'season_turnover_percent': 'opponent_season_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_turnover_percent, on=['opponent', 'season'], how='left')

In [308]:
season_turnover_percent_std = pre_tourney_games[['team', 'season','turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'turnover_percent': 'season_turnover_percent_std'})
tourney_games = tourney_games.merge(season_turnover_percent_std, on=['team', 'season'], how='left')
season_turnover_percent_std.rename(columns={'season_turnover_percent_std': 'opponent_season_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_turnover_percent_std, on=['opponent', 'season'], how='left')

In [309]:
vs_tourney_team_turnover_percent = games_vs_tourney_teams[['team', 'season','turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'turnover_percent': 'vs_tourney_team_turnover_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent, on=['team', 'season'], how='left')
vs_tourney_team_turnover_percent.rename(columns={'vs_tourney_team_turnover_percent': 'opponent_vs_tourney_team_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent, on=['opponent', 'season'], how='left')

In [310]:
vs_tourney_team_turnover_percent_std = games_vs_tourney_teams[['team', 'season','turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'turnover_percent': 'vs_tourney_team_turnover_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_turnover_percent_std.rename(columns={'vs_tourney_team_turnover_percent_std': 'opponent_vs_tourney_team_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent_std, on=['opponent', 'season'], how='left')

In [311]:
season_offensive_rebound_percent = pre_tourney_games[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_rebound_percent': 'season_offensive_rebound_percent'})
tourney_games = tourney_games.merge(season_offensive_rebound_percent, on=['team', 'season'], how='left')
season_offensive_rebound_percent.rename(columns={'season_offensive_rebound_percent': 'opponent_season_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [312]:
season_offensive_rebound_percent_std = pre_tourney_games[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_rebound_percent': 'season_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(season_offensive_rebound_percent_std, on=['team', 'season'], how='left')
season_offensive_rebound_percent_std.rename(columns={'season_offensive_rebound_percent_std': 'opponent_season_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [313]:
vs_tourney_team_offensive_rebound_percent = games_vs_tourney_teams[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_rebound_percent': 'vs_tourney_team_offensive_rebound_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent, on=['team', 'season'], how='left')
vs_tourney_team_offensive_rebound_percent.rename(columns={'vs_tourney_team_offensive_rebound_percent': 'opponent_vs_tourney_team_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [314]:
vs_tourney_team_offensive_rebound_percent_std = games_vs_tourney_teams[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_rebound_percent': 'vs_tourney_team_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_offensive_rebound_percent_std.rename(columns={'vs_tourney_team_offensive_rebound_percent_std': 'opponent_vs_tourney_team_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [315]:
season_free_throw_rate = pre_tourney_games[['team', 'season','free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'free_throw_rate': 'season_free_throw_rate'})
tourney_games = tourney_games.merge(season_free_throw_rate, on=['team', 'season'], how='left')
season_free_throw_rate.rename(columns={'season_free_throw_rate': 'opponent_season_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_free_throw_rate, on=['opponent', 'season'], how='left')

In [316]:
season_free_throw_rate_std = pre_tourney_games[['team', 'season','free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'free_throw_rate': 'season_free_throw_rate_std'})
tourney_games = tourney_games.merge(season_free_throw_rate_std, on=['team', 'season'], how='left')
season_free_throw_rate_std.rename(columns={'season_free_throw_rate_std': 'opponent_season_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [317]:
vs_tourney_team_free_throw_rate = games_vs_tourney_teams[['team', 'season','free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'free_throw_rate': 'vs_tourney_team_free_throw_rate'})
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate, on=['team', 'season'], how='left')
vs_tourney_team_free_throw_rate.rename(columns={'vs_tourney_team_free_throw_rate': 'opponent_vs_tourney_team_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate, on=['opponent', 'season'], how='left')

In [318]:
vs_tourney_team_free_throw_rate_std = games_vs_tourney_teams[['team', 'season','free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'free_throw_rate': 'vs_tourney_team_free_throw_rate_std'})
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate_std, on=['team', 'season'], how='left')
vs_tourney_team_free_throw_rate_std.rename(columns={'vs_tourney_team_free_throw_rate_std': 'opponent_vs_tourney_team_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [319]:
season_allowed_offensive_efficiency = pre_tourney_games[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_efficiency': 'season_allowed_offensive_efficiency'})
tournament_games = tourney_games.merge(season_allowed_offensive_efficiency, on=['team', 'season'], how='left')
season_allowed_offensive_efficiency.rename(columns={'season_allowed_offensive_efficiency': 'opponent_season_allowed_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_efficiency, on=['opponent', 'season'], how='left')

In [320]:
season_allowed_offensive_efficiency_std = pre_tourney_games[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_efficiency': 'season_allowed_offensive_efficiency_std'})
tourney_games = tourney_games.merge(season_allowed_offensive_efficiency_std, on=['team', 'season'], how='left')
season_allowed_offensive_efficiency_std.rename(columns={'season_allowed_offensive_efficiency_std': 'opponent_season_allowed_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [321]:
vs_tourney_team_allowed_offensive_efficiency = games_vs_tourney_teams[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_efficiency': 'vs_tourney_team_allowed_offensive_efficiency'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_efficiency.rename(columns={'vs_tourney_team_allowed_offensive_efficiency': 'opponent_vs_tourney_team_allowed_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency, on=['opponent', 'season'], how='left')

In [323]:
vs_tourney_team_allowed_offensive_efficiency_std = games_vs_tourney_teams[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_efficiency': 'vs_tourney_team_allowed_offensive_efficiency_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_efficiency_std.rename(columns={'vs_tourney_team_allowed_offensive_efficiency_std': 'opponent_vs_tourney_team_allowed_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [324]:
season_allowed_effective_fg_percent = pre_tourney_games[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_effective_fg_percent': 'season_allowed_effective_fg_percent'})
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent, on=['team', 'season'], how='left')
season_allowed_effective_fg_percent.rename(columns={'season_allowed_effective_fg_percent': 'opponent_season_allowed_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent, on=['opponent', 'season'], how='left')

In [325]:
season_allowed_effective_fg_percent_std = pre_tourney_games[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_effective_fg_percent': 'season_allowed_effective_fg_percent_std'})
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent_std, on=['team', 'season'], how='left')
season_allowed_effective_fg_percent_std.rename(columns={'season_allowed_effective_fg_percent_std': 'opponent_season_allowed_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [326]:
vs_tourney_team_allowed_effective_fg_percent = games_vs_tourney_teams[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_effective_fg_percent': 'vs_tourney_team_allowed_effective_fg_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent, on=['team', 'season'], how='left')
vs_tourney_team_allowed_effective_fg_percent.rename(columns={'vs_tourney_team_allowed_effective_fg_percent': 'opponent_vs_tourney_team_allowed_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent, on=['opponent', 'season'], how='left')

In [327]:
vs_tourney_team_allowed_effective_fg_percent_std = games_vs_tourney_teams[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_effective_fg_percent': 'vs_tourney_team_allowed_effective_fg_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_effective_fg_percent_std.rename(columns={'vs_tourney_team_allowed_effective_fg_percent_std': 'opponent_vs_tourney_team_allowed_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [328]:
season_allowed_turnover_percent = pre_tourney_games[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_turnover_percent': 'season_allowed_turnover_percent'})
tourney_games = tourney_games.merge(season_allowed_turnover_percent, on=['team', 'season'], how='left')
season_allowed_turnover_percent.rename(columns={'season_allowed_turnover_percent': 'opponent_season_allowed_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_turnover_percent, on=['opponent', 'season'], how='left')

In [329]:
season_allowed_turnover_percent_std = pre_tourney_games[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_turnover_percent': 'season_allowed_turnover_percent_std'})
tourney_games = tourney_games.merge(season_allowed_turnover_percent_std, on=['team', 'season'], how='left')
season_allowed_turnover_percent_std.rename(columns={'season_allowed_turnover_percent_std': 'opponent_season_allowed_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_turnover_percent_std, on=['opponent', 'season'], how='left')

In [330]:
vs_tourney_team_allowed_turnover_percent = games_vs_tourney_teams[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_turnover_percent': 'vs_tourney_team_allowed_turnover_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent, on=['team', 'season'], how='left')
vs_tourney_team_allowed_turnover_percent.rename(columns={'vs_tourney_team_allowed_turnover_percent': 'opponent_vs_tourney_team_allowed_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent, on=['opponent', 'season'], how='left')

In [331]:
vs_tourney_team_allowed_turnover_percent_std = games_vs_tourney_teams[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_turnover_percent': 'vs_tourney_team_allowed_turnover_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_turnover_percent_std.rename(columns={'vs_tourney_team_allowed_turnover_percent_std': 'opponent_vs_tourney_team_allowed_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent_std, on=['opponent', 'season'], how='left')

In [332]:
season_allowed_offensive_rebound_percent = pre_tourney_games[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'season_allowed_offensive_rebound_percent'})
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent, on=['team', 'season'], how='left')
season_allowed_offensive_rebound_percent.rename(columns={'season_allowed_offensive_rebound_percent': 'opponent_season_allowed_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [333]:
season_allowed_offensive_rebound_percent_std = pre_tourney_games[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'season_allowed_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent_std, on=['team', 'season'], how='left')
season_allowed_offensive_rebound_percent_std.rename(columns={'season_allowed_offensive_rebound_percent_std': 'opponent_season_allowed_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [334]:
vs_tourney_team_allowed_offensive_rebound_percent = games_vs_tourney_teams[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'vs_tourney_team_allowed_offensive_rebound_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_rebound_percent.rename(columns={'vs_tourney_team_allowed_offensive_rebound_percent': 'opponent_vs_tourney_team_allowed_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [335]:
vs_tourney_team_allowed_offensive_rebound_percent_std = games_vs_tourney_teams[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'vs_tourney_team_allowed_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_rebound_percent_std.rename(columns={'vs_tourney_team_allowed_offensive_rebound_percent_std': 'opponent_vs_tourney_team_allowed_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [336]:
season_allowed_free_throw_rate = pre_tourney_games[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_free_throw_rate': 'season_allowed_free_throw_rate'})
tourney_games = tourney_games.merge(season_allowed_free_throw_rate, on=['team', 'season'], how='left')
season_allowed_free_throw_rate.rename(columns={'season_allowed_free_throw_rate': 'opponent_season_allowed_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_free_throw_rate, on=['opponent', 'season'], how='left')

In [337]:
season_allowed_free_throw_rate_std = pre_tourney_games[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_free_throw_rate': 'season_allowed_free_throw_rate_std'})
tourney_games = tourney_games.merge(season_allowed_free_throw_rate_std, on=['team', 'season'], how='left')
season_allowed_free_throw_rate_std.rename(columns={'season_allowed_free_throw_rate_std': 'opponent_season_allowed_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [338]:
vs_tourney_team_allowed_free_throw_rate = games_vs_tourney_teams[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_free_throw_rate': 'vs_tourney_team_allowed_free_throw_rate'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate, on=['team', 'season'], how='left')
vs_tourney_team_allowed_free_throw_rate.rename(columns={'vs_tourney_team_allowed_free_throw_rate': 'opponent_vs_tourney_team_allowed_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate, on=['opponent', 'season'], how='left')

In [339]:
vs_tourney_team_allowed_free_throw_rate_std = games_vs_tourney_teams[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_free_throw_rate': 'vs_tourney_team_allowed_free_throw_rate_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_free_throw_rate_std.rename(columns={'vs_tourney_team_allowed_free_throw_rate_std': 'opponent_vs_tourney_team_allowed_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [340]:
tourney_games

,date,season,team,opponent,Win_Loss,regular_season_win_pct,opponent_regular_season_win_pct,season_win_pct,opponent_season_win_pct,season_ppg,...,vs_tourney_team_allowed_offensive_rebound_percent_std,opponent_vs_tourney_team_allowed_offensive_rebound_percent_std,season_allowed_free_throw_rate,opponent_season_allowed_free_throw_rate,season_allowed_free_throw_rate_std,opponent_season_allowed_free_throw_rate_std,vs_tourney_team_allowed_free_throw_rate,opponent_vs_tourney_team_allowed_free_throw_rate,vs_tourney_team_allowed_free_throw_rate_std,opponent_vs_tourney_team_allowed_free_throw_rate_std
0,3/17/23,2023,Arizona St.,TCU,L,0.645161,0.645161,0.647059,0.636364,70.323529,...,9.837169,7.243569,34.017647,30.539394,13.112554,11.704725,41.128571,31.342105,13.387217,12.412866
1,3/17/23,2023,TCU,Arizona St.,W,0.645161,0.645161,0.636364,0.647059,75.181818,...,7.243569,9.837169,30.539394,34.017647,11.704725,13.112554,31.342105,41.128571,12.412866,13.387217
2,3/23/23,2023,Arkansas,Connecticut,L,0.612903,0.774194,0.606061,0.757576,74.424242,...,10.429204,8.318787,39.651515,38.945455,15.859976,12.471620,44.409524,36.088889,14.042824,12.828089
3,3/23/23,2023,Connecticut,Arkansas,W,0.774194,0.612903,0.757576,0.606061,78.454545,...,8.318787,10.429204,38.945455,39.651515,12.471620,15.859976,36.088889,44.409524,12.828089,14.042824
4,3/18/23,2023,Arkansas,Kansas,W,0.612903,0.806452,0.606061,0.794118,74.424242,...,10.429204,9.762604,39.651515,31.544118,15.859976,13.034946,44.409524,31.234615,14.042824,12.978380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,3/19/15,2015,Wofford,Arkansas,L,0.793103,0.774194,0.812500,0.764706,65.875000,...,12.114537,11.733490,38.153125,39.347059,18.369768,15.406261,40.980000,44.550000,9.949724,16.126889
1238,3/20/15,2015,Northern Iowa,Wyoming,W,0.900000,0.689655,0.909091,0.718750,65.363636,...,8.244491,5.935992,26.939394,26.956250,12.266034,11.951216,32.672727,36.316667,10.988639,14.345092
1239,3/20/15,2015,Wyoming,Northern Iowa,L,0.689655,0.900000,0.718750,0.909091,61.437500,...,5.935992,8.244491,26.956250,26.939394,11.951216,12.266034,36.316667,32.672727,14.345092,10.988639
1240,3/26/15,2015,Arizona,Xavier,W,0.903226,0.612903,0.911765,0.617647,76.441176,...,8.705554,7.500967,36.726471,35.208824,14.133054,16.421777,41.068750,34.765000,14.598206,17.383182


In [341]:
tourney_games.to_sql('tourney_games_v6', conn, if_exists='replace', index=False, schema='sports_data')

154

In [199]:
conn.commit()